### Computing building density
This script will take the buildings from the area of study and perform a spatial analysis to calculate the density of features around a set of target points at different distances. The code iterates through each row of target_gdf one by one. For each row, it creates four buffers (50, 100, 250, 500) and counts how many features from all_gdf fall inside each buffer, saving those counts to new columns.

**Computational times**: The notebook was run on a local Machine with the following technical caracteristics: Processor	13th Gen Intel(R) Core(TM) i7-13700H (2.40 GHz), Installed RAM 16.0 GB (15.7 GB usable) and 64-bit operating system, x64-based processor. It was tested on Patan, India, which covers a total of 182.113 buildings. **The total processing time of this notebook was 95.88 minutes.**

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.wkt import loads
from shapely import wkb
import time

In [4]:
parquet_file = r"C:\Users\renec\OneDrive\Documents\SEForALL\GitHub\new-classification-model-local\Patan_Buildings.parquet"

In [6]:
target_df = pd.read_parquet(parquet_file)
target_df['geometry'] = target_df['geometry'].apply(wkb.loads)
target_gdf = gpd.GeoDataFrame(target_df, geometry='geometry', crs="EPSG:4326")
target_gdf = target_gdf.to_crs("EPSG:3857")

In [9]:
for dist in [50, 100, 250, 500]:
    target_gdf[f"building_density_{dist}"] = 0

In [8]:
starting_time = time.time()
batch_size = 1
results = []

for i in range(0, len(target_gdf), batch_size):
    #print(f"Processing batch {i}...")
    target_batch = target_gdf.iloc[i:i+batch_size].copy()

    for dist in [50, 100, 250, 500]:
        target_batch[f"building_density_{dist}"] = 0

        buffer_batch = target_batch.copy()
        buffer_batch["geometry"] = buffer_batch.geometry.buffer(dist)

        joined = gpd.sjoin(target_gdf, buffer_batch, how="inner", predicate="within")
        counts = joined.groupby("index_right").size()

        target_batch.loc[counts.index, f"building_density_{dist}"] = counts.values

    results.append(target_batch)

# concatenate all results
target_gdf = pd.concat(results)
target_gdf = target_gdf.to_crs("EPSG:4326")
target_gdf.to_parquet("try_AP_BD")

ending_time = time.time()

In [11]:
total_time = (ending_time - starting_time)/60
print(f"total elapse time:{round(total_time,2)} minutes")

total elapse time:95.88 minutes
